In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [2]:
# --- Load Tract-level data ---
tract_data = pd.read_csv('MH.csv')

# --- Load ZCTA crosswalk (tract to ZCTA relationship with RES_RATIO) ---
crosswalk = pd.read_csv('tract_to_zcta_crosswalk.csv')

In [3]:
# --- Check for data quality issues in tract_data ---
def check_data_quality(df, df_name):
    print(f"\n--- Data Quality Report for {df_name} ---")
    for col in df.columns:
        n_na = df[col].isna().sum()
        n_zero = (df[col] == 0).sum() if pd.api.types.is_numeric_dtype(df[col]) else 0
        n_blank = (df[col] == '').sum() if df[col].dtype == object else 0
        print(f"{col}: NaNs={n_na}, Zeros={n_zero}, Blanks={n_blank}")
        
check_data_quality(tract_data, "Tract Data")
check_data_quality(crosswalk, "Crosswalk")


--- Data Quality Report for Tract Data ---
GEOID: NaNs=0, Zeros=0, Blanks=0
MhCntDr: NaNs=0, Zeros=19546, Blanks=0
MhTmDr: NaNs=13987, Zeros=8002, Blanks=0
MhMinDis: NaNs=0, Zeros=0, Blanks=0

--- Data Quality Report for Crosswalk ---
TRACT: NaNs=0, Zeros=0, Blanks=0
ZIP: NaNs=0, Zeros=0, Blanks=0
USPS_ZIP_PREF_CITY: NaNs=0, Zeros=0, Blanks=0
USPS_ZIP_PREF_STATE: NaNs=0, Zeros=0, Blanks=0
RES_RATIO: NaNs=0, Zeros=11116, Blanks=0
BUS_RATIO: NaNs=0, Zeros=31532, Blanks=0
OTH_RATIO: NaNs=0, Zeros=57080, Blanks=0
TOT_RATIO: NaNs=0, Zeros=0, Blanks=0


In [4]:
# --- Merge tract data with crosswalk on tract GEOID ---
merged = crosswalk.merge(tract_data, left_on='TRACT', right_on='GEOID', how='left')

In [5]:
# --- Step 4: Weight access metrics by RES_RATIO ---
metrics = ['MhCntDr', 'MhTmDr', 'MhMinDis']
for var in metrics:
    if var in merged.columns:
        merged[f'{var}_wt'] = merged[var] * merged['TOT_RATIO']
    else:
        print(f"Warning: {var} not found in merged data!")

In [6]:
# --- Step 5: Quality check after merge ---
check_data_quality(merged, "Merged Data")


--- Data Quality Report for Merged Data ---
TRACT: NaNs=0, Zeros=0, Blanks=0
ZIP: NaNs=0, Zeros=0, Blanks=0
USPS_ZIP_PREF_CITY: NaNs=0, Zeros=0, Blanks=0
USPS_ZIP_PREF_STATE: NaNs=0, Zeros=0, Blanks=0
RES_RATIO: NaNs=0, Zeros=11116, Blanks=0
BUS_RATIO: NaNs=0, Zeros=31532, Blanks=0
OTH_RATIO: NaNs=0, Zeros=57080, Blanks=0
TOT_RATIO: NaNs=0, Zeros=0, Blanks=0
GEOID: NaNs=1, Zeros=0, Blanks=0
MhCntDr: NaNs=1, Zeros=44880, Blanks=0
MhTmDr: NaNs=26001, Zeros=17384, Blanks=0
MhMinDis: NaNs=1, Zeros=0, Blanks=0
MhCntDr_wt: NaNs=1, Zeros=44880, Blanks=0
MhTmDr_wt: NaNs=26001, Zeros=17384, Blanks=0
MhMinDis_wt: NaNs=1, Zeros=0, Blanks=0


In [7]:
# --- Step 6: Aggregate to ZIP level ---
zip_agg = merged.groupby('ZIP').agg({
    'MhCntDr_wt': 'sum',
    'MhTmDr_wt': 'sum',
    'MhMinDis_wt': 'sum',
    'TOT_RATIO': 'sum'
}).reset_index()

In [8]:
# --- Step 7: Calculate population-weighted averages ---
zip_agg['MhCntDr'] = zip_agg['MhCntDr_wt'] / zip_agg['TOT_RATIO']
zip_agg['MhTmDr'] = zip_agg['MhTmDr_wt'] / zip_agg['TOT_RATIO']
zip_agg['MhMinDis'] = zip_agg['MhMinDis_wt'] / zip_agg['TOT_RATIO']

In [9]:
# --- Step 8: Final output table ---
result = zip_agg[['ZIP', 'MhCntDr', 'MhTmDr', 'MhMinDis']]
print("\n--- Final ZIP-level Result Preview ---")
print(result.head())

# --- Step 9: Save result to CSV ---
result.to_csv('zipcode_weighted_Mh_2025.csv', index=False)


--- Final ZIP-level Result Preview ---
   ZIP  MhCntDr  MhTmDr   MhMinDis
0  501     50.0    4.79   0.590000
1  601      0.0    0.00  12.130874
2  602      0.0    0.00   7.541193
3  603      0.0    0.00   2.639544
4  604      0.0    0.00   2.400000


In [15]:
import pandas as pd

# === Step 1: Load Crosswalk and Tract-Level Data ===
crosswalk = pd.read_csv('tract_to_zcta_crosswalk.csv', dtype={'TRACT': str, 'ZIP': str})
tract_data = pd.read_csv('tract-2020.csv', dtype={'GEOID': str})

# Strip whitespace from column names
tract_data.columns = tract_data.columns.str.strip()

# === Step 2: Merge on GEOID and TRACT ===
merged = pd.merge(tract_data, crosswalk, left_on='GEOID', right_on='TRACT', how='left')

# Drop unmatched rows where RES_RATIO is missing
merged = merged.dropna(subset=['TOT_RATIO'])

# === Step 3: Identify columns to weight and aggregate ===
# Exclude identifier and geometry columns
cols_to_weight = [col for col in tract_data.columns if col != 'GEOID']

# Drop non-numeric columns if any (e.g., empty strings or weird text)
numeric_cols = merged[cols_to_weight].select_dtypes(include='number').columns.tolist()

# === Step 4: Weight each variable by RES_RATIO ===
for col in numeric_cols:
    merged[f'{col}_wt'] = merged[col] * merged['TOT_RATIO']

# === Step 5: Aggregate all weighted variables to ZIP level ===
agg_dict = {f'{col}_wt': 'sum' for col in numeric_cols}
agg_dict['TOT_RATIO'] = 'sum'

zip_agg = merged.groupby('ZIP').agg(agg_dict).reset_index()

# === Step 6: Compute final weighted average ===
for col in numeric_cols:
    zip_agg[col + '_ZIP'] = zip_agg[f'{col}_wt'] / zip_agg['TOT_RATIO']

# === Step 7: Keep only ZIP and final weighted columns ===
output_cols = ['ZIP'] + [col + '_ZIP' for col in numeric_cols]
final_result = zip_agg[output_cols]

# === Step 8: Export to CSV ===
output_path = 'zipcode_weighted_tract_2019.csv'
final_result.to_csv(output_path, index=False)
print(f"✅ Saved ZIP-level weighted data to: {output_path}")


/tmp/ipykernel_1185/3492424692.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged[f'{col}_wt'] = merged[col] * merged['TOT_RATIO']
/tmp/ipykernel_1185/3492424692.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged[f'{col}_wt'] = merged[col] * merged['TOT_RATIO']
/tmp/ipykernel_1185/3492424692.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.

✅ Saved ZIP-level weighted data to: zipcode_weighted_tract_2019.csv
